In [1]:
import pandas as pd
import tqdm

import re
import requests
from bs4 import BeautifulSoup

import hashlib
import datetime

In [2]:
base_url = 'http://www.curitiba.pr.gov.br/dadosabertos/consulta'

In [3]:
params = {"grupo":"2"}

In [4]:
r = requests.get(base_url, params=params)

In [5]:
page = BeautifulSoup(r.text, 'html.parser')

In [6]:
__EVENTVALIDATION = page.find("input", {'name': '__EVENTVALIDATION'}).get('value')

In [7]:
__EVENTVALIDATION

'/wEdAAyR7/BxHIGQUcWKBteuCXEG4mKL5o1QRo/FpCKPmYsWJJ6M545mi+eVxYOsufjJuPs5znhzSWYfXU4ugll/AJGuS7DlCF7QoIFOF8dSWE2wXzkiLQS8NWTPNZDV80Z3AzzphndoyDFnIVED7A3KQw9cs48DT8q4XVH83r8Yw95EbJ8/8wAqn890Z+wGGuGKtYqNvjaF5GeBQ6wPsZcNPGG+vCOATgFUUjl7PbiZupqgZarqqZvleoeig3SW/qIZQc30b5s8YAzVC8Lrkabyd4HTPEeQIQ=='

In [8]:
__VIEWSTATE = page.find("input", {'name': '__VIEWSTATE'}).get('value')

In [9]:
__VIEWSTATE

'/wEPDwUJMzc0Mzc1MzA2DxYGHgtzdGF0ZV9ncnVwbwUBMh4IaWRmR3J1cG8CAh4IYnVzY2FQb3JkFgJmD2QWAgIDD2QWAgIBDxYCHgZhY3Rpb24FHi9kYWRvc2FiZXJ0b3MvY29uc3VsdGE/Z3J1cG89MhYOAgIPEA8WBh4NRGF0YVRleHRGaWVsZAUKTm9tZVBhZ2luYR4ORGF0YVZhbHVlRmllbGQFCVVybFBhZ2luYR4LXyFEYXRhQm91bmRnFgQeCW9ua2V5ZG93bgVDaWYoSXNFbnRlcktleShldmVudCkpe1JlZGlyZWNpb25hckFjZXNzb1JhcGlkbyh0aGlzKTtyZXR1cm4gZmFsc2U7fR4Hb25jbGljawUfUmVkaXJlY2lvbmFyQWNlc3NvUmFwaWRvKHRoaXMpOxAVBg5BY2Vzc28gUsOhcGlkbxZTZWNyZXRhcmlhcyBlIMOTcmfDo29zEVNlcnZpw6dvcyBPbi1MaW5lGFNlcnZpw6dvcyBwYXJhIGEgRW1wcmVzYRlTZXJ2acOnb3MgcGFyYSBvIENpZGFkw6NvE1NpdGVzIGRhIFByZWZlaXR1cmEVBgANL3NlY3JldGFyaWFzLxkvbGlzdGEtc2Vydmljb3Mtb24tbGluZS8yES9zZXJ2aWNvcy9lbXByZXNhES9zZXJ2aWNvcy9jaWRhZGFvHC9saXN0YS1zaXRlcy1kYS1wcmVmZWl0dXJhLzUUKwMGZ2dnZ2dnZGQCBA8PZBYCHwcFSXJldHVybiBWYWxpZGFyUGVzcXVpc2EoJ3R4dFBlc3F1aXNhVG9wbycsJ2N0bDAwJGxua1Blc3F1aXNhVG9wbycsIGV2ZW50KTtkAgYPDxYCHgRUZXh0BQ1TZXgsIDEyLzEwLzE4ZGQCBw8PFgIeCEltYWdlVXJsBRV+L2ltYWdlbS90ZW1wby9wYy5naWZkZAIIDw8WAh8JBQQyNcK6ZGQCCQ8PFgIfCQUEMTX

In [10]:
pagination = page.find_all(class_='paginacao2')
pagination = list(pagination[0].text.replace('\n', ''))

In [11]:
pages = ""
for page in pagination:
    data = {
        "__EVENTTARGET": f"ctl00$cphMasterPortal$ucPaginador${page}_pg{page}",
        "__EVENTVALIDATION": __EVENTVALIDATION,
        "__VIEWSTATE": __VIEWSTATE
    }
    
    r = requests.request("POST", base_url, params=params, data=data)
    pages = pages + r.text

In [12]:
md5 = lambda s : hashlib.md5(s.encode('utf-8')).hexdigest()

In [13]:
md5('test')

'098f6bcd4621d373cade4e832627b4f6'

In [14]:
a_tags = BeautifulSoup(pages, 'html.parser').find_all(id='linkArquivo')

In [15]:
print(md5(a_tags[0].text))

167842ae181d01226901c42b20520afe


In [16]:
date_regex = re.compile('\d{4}-\d{2}-\d{2}')

In [17]:
date_string = date_regex.search('//multimidia.curitiba.pr.gov.br/dadosabertos/SESPAMedicoUnidadesMunicipaisDeSaude/2018-10-10_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv').group()
date = datetime.date(*map(int, date_string.split('-')))
date


datetime.date(2018, 10, 10)

In [18]:
databases = []
for a_tag in tqdm.tqdm(a_tags):
    database_link = a_tag.get('href')[2:]
    
    if database_link[-4:] == '.csv':
        database_id = md5(a_tag.text)
        database_title = a_tag.text

        database_date_string = date_regex.search(database_link).group()
        database_date = datetime.date(*map(int, database_date_string.split('-')))

        database = {
            'database_id': database_id,
            'database_title': database_title,
            'database_link': database_link,
            'database_date': database_date
        }
        
        databases.append(database)

100%|██████████| 10/10 [00:00<00:00, 14644.92it/s]


In [19]:
databases

[{'database_id': '91b5d868a8ec96fc18d3d3f0e322a804',
  'database_title': 'Alvaras-Dicionario de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/BaseAlvaras/2015-05-06_Alvaras-Dicionario_de_Dados.csv',
  'database_date': datetime.date(2015, 5, 6)},
 {'database_id': 'f7e8a635bfa59039b7853027a2956c85',
  'database_title': 'Despesas - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/BaseReceitaDespesa/2018-10-01_Despesas_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 1)},
 {'database_id': 'd9a2b67fd5f1799e443d1971b585f946',
  'database_title': 'Receitas - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/BaseReceitaDespesa/2018-10-01_Receitas_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 1)}]

In [26]:
test = pd.read_csv('http://multimidia.curitiba.pr.gov.br/dadosabertos/BaseReceitaDespesa/2018-10-01_Receitas_-_Base_de_Dados.csv', encoding="ISO-8859-1", delimiter=';')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,3,5,7,8,9,10,11,12,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
test.fillna('').head()

,CD_RECEITA,CD_CATEGORIA,DESCRICAO_CATEGORIA,CD_ORIGEM,DESCRICAO_ORIGEM,CD_ESPECIE,DESCRICAO_ESPECIE,CD_RUBRICA,DESCRICAO_RUBRICA,CD_ALINEA,DESCRICAO_ALINEA,CD_SUBALINEA,DESCRICAO_SUBALINEA,CD_EXERCICIO,DT_APROPRIACAO,TP_RECEITA_ORC,CD_EMPRESA,NM_EMPRESA,VL_RECEITA,DS_FONTE
0,----------,------------,-------------------,---------,----------------,----------,-----------------,----------,-----------------,---------,----------------,------------,-------------------,------------,--------------,--------------,----------,----------,----------,--------
1,1122011201000000000000,1,RECEITAS CORRENTES,1,"IMPOSTOS, TAXAS E CONTRIBUIÇÕES DE MELHORIA",2,TAXAS,,,,,,,2018,2018-08-20,Orçamentária,1,Prefeitura Municipal de Curitiba,"946,27",Taxas - Prestação de Serviços
2,1122011201000000000000,1,RECEITAS CORRENTES,1,"IMPOSTOS, TAXAS E CONTRIBUIÇÕES DE MELHORIA",2,TAXAS,,,,,,,2018,2018-08-20,Orçamentária,1,Prefeitura Municipal de Curitiba,"1,76",Taxas - Prestação de Serviços
3,1122011301000000000000,1,RECEITAS CORRENTES,1,"IMPOSTOS, TAXAS E CONTRIBUIÇÕES DE MELHORIA",2,TAXAS,,,,,,,2018,2018-08-20,Orçamentária,1,Prefeitura Municipal de Curitiba,"78,87",Taxas - Prestação de Serviços
4,1122011301000000000000,1,RECEITAS CORRENTES,1,"IMPOSTOS, TAXAS E CONTRIBUIÇÕES DE MELHORIA",2,TAXAS,,,,,,,2018,2018-08-20,Orçamentária,1,Prefeitura Municipal de Curitiba,"93,19",Taxas - Prestação de Serviços
